In [1]:
import numpy as np
import matplotlib.pyplot as plt
import fitsio

In [2]:
from astropy.cosmology import FlatLambdaCDM
Om = 0.31519
cosmo = FlatLambdaCDM(H0=100, Om0=Om)
fsky = 1437./41253. #(1321 + 116) 
# zmin = 0.2
# zmax = 0.35
zmin = 0.35
zmax = 0.50 # TODO: make a dictionary

vol_des = cosmo.comoving_volume(zmax).value - cosmo.comoving_volume(zmin).value
vol_des = vol_des * fsky
print('DES volume = %e (Mpc/h)^3'%vol_des)
print('DES volume = %g (Gpc/h)^3'%(vol_des*1e-9))
print('DES volume = (%g Mpc/h)^3'%(vol_des**(1./3.)))

DES volume = 2.031314e+08 (Mpc/h)^3
DES volume = 0.203131 (Gpc/h)^3
DES volume = (587.84 Mpc/h)^3


In [3]:
#### DES Y1 counts ####
loc = '/users/hywu/important_data/DES-Y1/'
fname = loc + 'y1a1_gold_1.0.3-d10-mof-001d_run_redmapper_v6.4.17_lgt5_desformat_catalog.fit'
import fitsio
data, header = fitsio.read(fname, header=True)
#print(header)
zlam = data['Z_LAMBDA']
lam = data['LAMBDA_CHISQ']

# first verify with DES Y1 CL paper
lam_min = np.array([20, 30, 45, 60])
lam_max = np.array([30, 45, 60, 1000])

nbin = len(lam_min)
NC_list = []
for ibin in range(nbin):
    sel = (zlam > zmin)&(zlam < zmax)&(lam >= lam_min[ibin])&(lam < lam_max[ibin])
    
    print('mean z =',np.mean(zlam[sel]))
    print('median z =',np.median(zlam[sel]))
    NC = len(lam[sel])
    print('NC=', NC)
    NC_list.append(NC)
## Yes, it agrees!

mean z = 0.42959964
median z = 0.42829236
NC= 1549
mean z = 0.42609707
median z = 0.4215436
NC= 672
mean z = 0.43071392
median z = 0.42749658
NC= 187
mean z = 0.42906234
median z = 0.42768365
NC= 148


In [4]:
# cum: start from the richest
NC_cum = np.cumsum(NC_list[::-1])[::-1]
print(NC_list)
print(NC_cum)
out_loc = 'data/'
np.savetxt(out_loc+f'cluster_cumulative_counts_no_miscen_z_{zmin}_{zmax}.dat', 
           NC_cum, fmt='%g', header='no miscentering')

[1549, 672, 187, 148]
[2556 1007  335  148]


In [5]:
# calculate the cumulative density (for matching space density)
import numpy as np
lam_min = np.array([20, 30, 45, 60])
lam_max = np.array([30, 45, 60, 1000])
# TODO: need other z values
# NC = np.array([785.1, 388.3, 127.2, 93.9])  # Table 1, miscen corrected
# dNC_stat = np.array([54.9, 32.1, 15.2, 14.0]) 
space_density = NC/vol_des
print('space density in bins', space_density)

cum_num = []
for i in range(4):
    cum_num.append(np.sum(space_density[i:]))

cum_num2 = np.cumsum(space_density)
out_loc = 'data/'
np.savetxt(out_loc+f'cluster_cumulative_density_z_{zmin}_{zmax}.dat', 
           cum_num, fmt='%-e', header='h^3 Mpc-3 (Om=%g)'%(Om))

space density in bins [3.86498550e-06 1.91157033e-06 6.26195587e-07 4.62262308e-07]


In [6]:
# vol_abacus = 1100**3
# cum_num_abacus = np.array(np.around(cum_num * vol_abacus)+1e-4, dtype=int)
# print('cum number in abacus', cum_num_abacus)

In [7]:
# vol_summit = 2000**3
# cum_num_summit = np.array(np.around(cum_num * vol_summit)+1e-4, dtype=int)
# print('cum number in abacus summit', cum_num_summit)

In [8]:
# vol_mini = 400**3
# cum_num_mini = np.array(np.around(cum_num * vol_mini)+1e-4, dtype=int)
# print('cum number in mini uchuu', cum_num_abacus)

In [9]:
# ## calculate the corresponding mass asssuming there's no scatter
# # read abacus 
# import sys
# sys.path.append('../utils/')
# from read_abacus_summit import ReadAbacusSummit
# nbody_loc = '/bsuhome/hwu/scratch/abacus_summit/'
# ras = ReadAbacusSummit(nbody_loc)
# ras.read_halos(1e13)
# mass_summit = ras.mass

In [10]:
# # add 0.2 scatter
# scatt = 0.2
# ln_mass_summit = np.log(mass_summit)
# nhalo = len(mass_summit)
# ln_mass_summit += np.random.normal(0, scatt, nhalo)
# ln_mass_summit = np.sort(ln_mass_summit)[::-1]
# bin_edges =  np.exp(ln_mass_summit[cum_num_summit])
# print('abundance matched mass bin edges', np.exp(ln_mass_summit[cum_num_summit]))
# Mmin_obs = bin_edges
# Mmax_obs = np.append(bin_edges[1:], 1e16)
# data = np.array([Mmin_obs, Mmax_obs]).transpose()
# zmin = 0.2
# np.savetxt(f'data/Mobs_bins_cov_z{zmin}_scatt{scatt}.dat', data, fmt='%-12g', header='Mobs_min, Mobs_max')

In [11]:
# # calculate the cumulative space density vs. richness
# lam_min_list = 10**np.linspace(np.log10(20), np.log10(100), 20)
# den_list = []
# for lam_min in lam_min_list:
#     sel = (lam >= lam_min)
#     den_list.append(len(lam[sel]))
# den_list = np.array(den_list)/vol_des
# plt.plot(lam_min_list, den_list)
# plt.loglog()

# from scipy.interpolate import interp1d
# lam_interp = interp1d(den_list[::-1], lam_min_list[::-1])
# print(lam_interp(1e-5))
# print(lam_interp(3e-6))
# print(lam_interp(1e-6))

In [12]:
# h^3 Mpc-3 (Om=0.31519)
1.371301e-05
5.992619e-06
2.174217e-06
9.233786e-07

9.233786e-07